In [5]:

import numpy as np
import sys
sys.path.append("..")
from pygalaxy.d_opengl import Animation
from pygalaxy.d_forces import force
from pygalaxy.d_init import init_solar_system
                    
planets = ['sun', 'mercury', 'venus', 'earth', 'mars']


# Method for calculating forces
def compute_forces(mass, particles, output, dim=3):
    n = particles.shape[0]
    output.fill(0)
    for i in range(n):
        for j in range(n):
            if i != j:
                f = force(particles[i, :dim], particles[j, :dim], mass[j], dim)
                output[i, :dim] += f
    output[:, dim:] = particles[:, dim:]

# Symplectic Euler integrator
def euler_symplectic(dt, mass, particles, dim=3):
    forces = np.zeros_like(particles)
    compute_forces(mass, particles, forces, dim)
    particles[:, dim:] += dt * forces[:, dim:]
    compute_forces(mass, particles, forces, dim)
    particles[:, :dim] += dt * forces[:, :dim]

# Main function
if __name__ == '__main__':
    dim = 2
    mass, bodies = init_solar_system(dim=dim)
    dt = 60 * 60  # One hour timestep

    # Wrap the simulation in a class
    class SolarSystemSim:
        def __init__(self, mass, bodies, dt):
            self.mass = mass
            self.particles = bodies
            self.dt = dt

        def next(self):
            euler_symplectic(self.dt, self.mass, self.particles, dim=dim)

        def coords(self):
            return self.particles[:, :, 0]

    # Initialize simulation and animation
    simu = SolarSystemSim(mass, bodies, dt)
    anim = Animation(simu, axis=[-1.5e11, 1.5e11, -1.5e11, 1.5e11, -1.5e11, 1.5e11], size=[800, 800], dim=dim)
    anim.main_loop()


NullFunctionError: Attempt to call an undefined function glutInitDisplayMode, check for bool(glutInitDisplayMode) before calling